# Traffic Sign Classification — GTSRB

> Jupyter Notebook to classify German Traffic Signs (GTSRB) using a custom CNN and transfer learning (MobileNetV2). Designed to run on a local CUDA-enabled laptop.

**What you'll get:** dataset loading, preprocessing, augmentation, custom CNN training, evaluation (accuracy + confusion matrix), and a MobileNetV2 transfer-learning comparison.

---

**Quick instructions:**
- Update `DATA_ROOT` to point to your extracted Kaggle dataset. The notebook expects folders like `Final_Training/Images` within that root.
- Make sure you have a TensorFlow GPU build installed compatible with your CUDA/CuDNN.
- Run cells in order.

## 1. Imports & GPU check

In [ ]:
# Basic imports and GPU check
import os
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print("TensorFlow version:", tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
print('GPUs found:', gpus)
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

## 2. Paths / Dataset structure
Adjust these paths to where you extracted your Kaggle dataset.

In [ ]:
# Update this path to where you extracted the Kaggle GTSRB dataset
DATA_ROOT = './gtsrb'  # change to your dataset root
TRAIN_DIR = os.path.join(DATA_ROOT, 'Final_Training', 'Images')
TEST_DIR = os.path.join(DATA_ROOT, 'Final_Test', 'Images')

print('Train dir exists?', os.path.exists(TRAIN_DIR))
print('Test dir exists?', os.path.exists(TEST_DIR))

## 3. Load dataset (resizing + split)
We'll use image_dataset_from_directory and split a validation set from training data.

In [ ]:
BATCH_SIZE = 64
IMG_SIZE = (64, 64)
VAL_SPLIT = 0.15
SEED = 123

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=VAL_SPLIT,
    subset='training',
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='int'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=VAL_SPLIT,
    subset='validation',
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='int'
)

class_names = train_ds.class_names
NUM_CLASSES = len(class_names)
print('Found classes:', NUM_CLASSES)
print('Class names (first 10):', class_names[:10])

## 4. Prefetch, Caching, and Normalization

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

normalization_layer = layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=AUTOTUNE)

# Show a batch sample
for images, labels in train_ds.take(1):
    print('Batch image shape:', images.shape)
    print('Batch labels shape:', labels.shape)
    break

## 5. Data Augmentation
We'll build a simple augmentation pipeline and visualize samples.

In [ ]:
data_augmentation = keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.08),
    layers.RandomZoom(0.08),
    layers.RandomTranslation(0.05, 0.05)
], name='data_augmentation')

# Visualize augmentation example
for images, labels in train_ds.take(1):
    plt.figure(figsize=(9,9))
    augmented = data_augmentation(images)
    for i in range(9):
        ax = plt.subplot(3,3,i+1)
        plt.imshow(augmented[i].numpy())
        plt.axis('off')
    plt.suptitle('Augmented samples')
    plt.show()
    break

## 6. Custom CNN Model

In [ ]:
def make_custom_cnn(input_shape=(64,64,3), num_classes=NUM_CLASSES):
    inputs = keras.Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Flatten()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs, outputs, name='custom_cnn')
    return model

model = make_custom_cnn()
model.summary()

## 7. Compile and Train (Custom CNN)

In [ ]:
EPOCHS = 30

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    keras.callbacks.ModelCheckpoint('best_custom_cnn.h5', save_best_only=True, monitor='val_accuracy'),
    keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=6, restore_best_weights=True)
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)

## 8. Evaluate & Plot

In [ ]:
def plot_history(history):
    fig, axes = plt.subplots(1,2, figsize=(14,4))
    axes[0].plot(history.history['loss'], label='train_loss')
    axes[0].plot(history.history['val_loss'], label='val_loss')
    axes[0].legend(); axes[0].set_title('Loss')

    axes[1].plot(history.history['accuracy'], label='train_acc')
    axes[1].plot(history.history['val_accuracy'], label='val_acc')
    axes[1].legend(); axes[1].set_title('Accuracy')

plot_history(history)

## 9. Confusion Matrix & Classification Report

In [ ]:
# Collect labels and predictions on the validation set
y_true = []
y_pred = []

for images, labels in val_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

cm = confusion_matrix(y_true, y_pred)
print(classification_report(y_true, y_pred, target_names=class_names))

# Plot normalized confusion matrix
plt.figure(figsize=(12,10))
cm_norm = cm.astype('float') / (cm.sum(axis=1)[:, np.newaxis] + 1e-9)
sns.heatmap(cm_norm, annot=False, cmap='viridis')
plt.title('Normalized Confusion Matrix (validation)')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

## 10. Bonus: Transfer Learning with MobileNetV2
We'll prepare datasets at a larger size and train a MobileNetV2 head.

In [ ]:
IMG_SIZE_TL = (96, 96)
BATCH_SIZE_TL = 32

train_ds_tl = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=VAL_SPLIT,
    subset='training',
    seed=SEED,
    image_size=IMG_SIZE_TL,
    batch_size=BATCH_SIZE_TL,
    label_mode='int'
)
val_ds_tl = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=VAL_SPLIT,
    subset='validation',
    seed=SEED,
    image_size=IMG_SIZE_TL,
    batch_size=BATCH_SIZE_TL,
    label_mode='int'
)

train_ds_tl = train_ds_tl.map(lambda x, y: (tf.cast(x, tf.float32)/255.0, y)).cache().prefetch(AUTOTUNE)
val_ds_tl = val_ds_tl.map(lambda x, y: (tf.cast(x, tf.float32)/255.0, y)).cache().prefetch(AUTOTUNE)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SIZE_TL + (3,), include_top=False, weights='imagenet')
base_model.trainable = False

inputs = keras.Input(shape=IMG_SIZE_TL + (3,))
x = data_augmentation(inputs)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model_tl = keras.Model(inputs, outputs, name='mobilenetv2_tl')
model_tl.compile(optimizer=keras.optimizers.Adam(1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_tl.summary()

callbacks_tl = [keras.callbacks.ModelCheckpoint('best_mobilenetv2.h5', save_best_only=True, monitor='val_accuracy'),
                keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)]

history_tl = model_tl.fit(train_ds_tl, validation_data=val_ds_tl, epochs=12, callbacks=callbacks_tl)

### Fine-tuning (optional)
Unfreeze top layers and fine-tune with a lower learning rate.

In [ ]:
# Example fine-tune
base_model.trainable = True
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model_tl.compile(optimizer=keras.optimizers.Adam(1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history_ft = model_tl.fit(train_ds_tl, validation_data=val_ds_tl, epochs=8, callbacks=callbacks_tl)

## 11. Compare results

In [ ]:
print('Custom CNN best val_acc:', max(history.history['val_accuracy'])))
print('MobileNetV2 best val_acc:', max(history_tl.history['val_accuracy']))

## 12. Save models & tips

In [ ]:
# Save final models
model.save('final_custom_cnn_saved')
model_tl.save('final_mobilenetv2_saved')

print('Models saved to disk.')